# Kazakhstan urbanization review

In support of Giuseppe Rossitti and Tom Farole, we need to generate a set of standard urban analyses

1. EC urban clusters using WorldPop 2020
   - need to name the clusters as well
2. Nighttime Lights for #1
3. GHSL for #1
4. Flood risk
5. Variation in precipitation and temperature
6. Air quality (PM 2.5)

In [ ]:
import sys
import os
import importlib
import json
import boto3
import rasterio
import folium

import pandas as pd
import geopandas as gpd
import numpy as np
import skimage.graph as graph

from botocore.config import Config
from botocore import UNSIGNED
from shapely.geometry import Point, mapping
from scipy import ndimage

sys.path.insert(0, "/home/wb411133/Code/GOSTNets_Raster/src")
import GOSTNetsRaster.market_access as ma
# import GOSTNetsRaster.conversion_tables as speed_tables

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.ntlMisc as ntl
from GOSTrocks.misc import tPrint

sys.path.append("../../../src")
import GOST_Urban.UrbanRaster as urban
import GOST_Urban.urban_helper as clippy

%load_ext autoreload
%autoreload 2

# read in local important parameters
local_json = "/home/wb411133/Code/urbanParameters.json"
with open(local_json, "r") as inJ:
    important_vars = json.load(inJ)

s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

In [ ]:
# Define input data
iso3 = "KAZ"
global_population_file = (
    "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
)
global_population_ghs_file = "/home/public/Data/GLOBAL/Population/GHS/2022_1km/GHS_POP_E2020_GLOBE_R2022A_54009_1000_V1_0.tif"
global_ghsl_folder = "/home/public/Data/GLOBAL/GHSL/v2022/"
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
admin_bounds = "/home/public/Data/COUNTRY/KAZ/ADMIN/kaz_districts.shp"

output_folder = "/home/wb411133/projects/KAZ_SCADR_Urbanization"
output_data = os.path.join(output_folder, "DATA")
worldpop_urban = os.path.join(output_data, "WorldPop_Urban")
ghspop_urban = os.path.join(output_data, "GHS_Urban")
ghsl_folder = os.path.join(output_data, "GHSL")
ma_folder = os.path.join(output_data, "MARKET_ACCESS")

if not os.path.exists(ma_folder):
    os.makedirs(ma_folder)

# Define output files
local_population = os.path.join(output_data, f"{iso3}_ppp_2020_1km_aggregated.tif")
local_ghs_population = os.path.join(output_data, f"{iso3}_ghs_pop_2020.tif")
local_friction = os.path.join(output_data, f"{iso3}_2020_motorized_travel.tif")
urban_tt_result = os.path.join(output_data, "urban_travel_time.csv")

"""
urban_extents_file        = os.path.join(worldpop_urban, f"{iso3}_urban_extents.geojson")
urban_extents_raster_file = os.path.join(worldpop_urban, f"{iso3}_urban_extents.tif")
urban_extents_hd_file     = os.path.join(worldpop_urban, f"{iso3}_urban_extents_hd.geojson")
urban_extents_hd_raster_file = os.path.join(worldpop_urban, f"{iso3}_urban_extents_hd.tif")
admin_urban_summary       = os.path.join(worldpop_urban, "adm2_urban_summary.shp")
urban_admin_summary       = os.path.join(worldpop_urban, f"{iso3}_ADM2_urban_summary.csv")
"""
urban_extents_file = os.path.join(ghspop_urban, f"{iso3}_urban_extents.geojson")
urban_extents_raster_file = os.path.join(ghspop_urban, f"{iso3}_urban_extents.tif")
urban_extents_hd_file = os.path.join(ghspop_urban, f"{iso3}_urban_extents_hd.geojson")
urban_extents_hd_raster_file = os.path.join(
    ghspop_urban, f"{iso3}_urban_extents_hd.tif"
)
admin_urban_summary = os.path.join(ghspop_urban, "adm2_urban_summary.shp")
urban_admin_summary = os.path.join(ghspop_urban, f"{iso3}_ADM2_urban_summary.csv")

urban_viirs_summary = os.path.join(output_folder, f"{iso3}_urban_viirs_summary.csv")
urban_hd_viirs_summary = os.path.join(
    output_folder, f"{iso3}_urban_hd_viirs_summary.csv"
)
admin_viirs_summary = os.path.join(output_folder, f"{iso3}_admin_viirs_summary.csv")

urban_ghsl_summary = os.path.join(output_folder, f"{iso3}_urban_ghsl_summary.csv")
urban_hd_ghsl_summary = os.path.join(output_folder, f"{iso3}_urban_hd_ghsl_summary.csv")
admin_ghsl_summary = os.path.join(output_folder, f"{iso3}_admin_ghsl_summary.csv")

admin_final = os.path.join(output_folder, "admin_summarized.shp")
urban_final = os.path.join(output_folder, "urban_summarized.shp")
urban_hd_final = os.path.join(output_folder, "urban_hd_summarized.shp")
focal_cities = os.path.join(output_folder, "FOCAL_AOIs.shp")

# Define market access output
all_routes_file = os.path.join(ma_folder, "all_routes.shp")
time_matrix = os.path.join(ma_folder, "all_routes_time_minutes.csv")
dist_matrix = os.path.join(ma_folder, "all_routes_distance_km.csv")
dist_all_routes_file = os.path.join(ma_folder, "all_routes.shp")
dist_time_matrix = os.path.join(ma_folder, "district_routes_time_minutes.csv")
dist_dist_matrix = os.path.join(ma_folder, "district_routes_distance_km.csv")

In [ ]:
inAdmin = gpd.read_file(admin_bounds)

if not os.path.exists(local_population):
    globalP = rasterio.open(global_population_file)
    rMisc.clipRaster(globalP, inAdmin, local_population)

if not os.path.exists(local_ghs_population):
    globalP = rasterio.open(global_population_ghs_file)
    rMisc.clipRaster(globalP, inAdmin, local_ghs_population)

inP = rasterio.open(local_population)
inP_ghs = rasterio.open(local_ghs_population)

# Run urbanization analysis
1. Create urban extents  
2. Calculate urban population in admin bounds  
3. Summarize nighttime lights in extents and admin
4. Summarize GHSL in extents and admin

In [ ]:
# 1. Create urban extents for WorldPop
if not os.path.exists(urban_extents_file):
    urban_calculator = urban.urbanGriddedPop(inP)
    urban_extents = urban_calculator.calculateUrban(
        densVal=300,
        totalPopThresh=5000,
        smooth=False,
        queen=False,
        verbose=True,
        raster=urban_extents_raster_file,
    )
    urban_extents_hd = urban_calculator.calculateUrban(
        densVal=1500,
        totalPopThresh=50000,
        smooth=True,
        queen=False,
        verbose=True,
        raster=urban_extents_hd_raster_file,
    )
    # Name urban extents
    urban_extents = urban.geocode_cities(urban_extents)
    urban_extents_hd = urban.geocode_cities(urban_extents_hd)
    urban_extents.to_file(urban_extents_file, driver="GeoJSON")
    urban_extents_hd.to_file(urban_extents_hd_file, driver="GeoJSON")
else:
    urban_extents = gpd.read_file(urban_extents_file)
    urban_extents_hd = gpd.read_file(urban_extents_hd_file)

In [ ]:
# 1b. Create urban extents for GHS_Pop
if not os.path.exists(urban_extents_file):
    urban_calculator = urban.urbanGriddedPop(inP_ghs)
    urban_extents = urban_calculator.calculateUrban(
        densVal=300,
        totalPopThresh=5000,
        smooth=False,
        queen=False,
        verbose=True,
        raster=urban_extents_raster_file,
    )
    if urban_extents.crs.to_epsg() != 4326:
        urban_extents = urban_extents.to_crs(4326)
    urban_extents = urban.geocode_cities(urban_extents)
    urban_extents.to_file(urban_extents_file, driver="GeoJSON")
if not os.path.exists(urban_extents_hd_file):
    urban_extents_hd = urban_calculator.calculateUrban(
        densVal=1500,
        totalPopThresh=50000,
        smooth=True,
        queen=False,
        verbose=True,
        raster=urban_extents_hd_raster_file,
    )
    if urban_extents_hd.crs.to_epsg() != 4326:
        urban_extents_hd = urban_extents_hd.to_crs(4326)
    # Name urban extents
    urban_extents_hd = urban.geocode_cities(urban_extents_hd)
    urban_extents_hd.to_file(urban_extents_hd_file, driver="GeoJSON")
else:
    urban_extents = gpd.read_file(urban_extents_file)
    urban_extents_hd = gpd.read_file(urban_extents_hd_file)

In [ ]:
# 2. Calculate urban population in admin areas
if not os.path.exists(urban_admin_summary):
    pop_worker = clippy.summarize_population(
        local_ghs_population,
        inAdmin,
        urban_extents_raster_file,
        urban_extents_hd_raster_file,
    )
    summarized_urban = pop_worker.calculate_zonal()
    urban_res = summarized_urban.loc[
        :, [x for x in summarized_urban.columns if "SUM" in x]
    ]
    urban_res.columns = ["TOTAL_POP", "URBAN_POP", "URBAN_HD_POP"]
    urban_res["district_c"] = inAdmin["district_c"]
    urban_res["district"] = inAdmin["district"]
    urban_res.to_csv(urban_admin_summary)

In [ ]:
# 3. summarize nighttime lights
ntl_files = ntl.aws_search_ntl()

viirs_folder = os.path.join(output_data, "NTL_ZONAL_RES")
if not os.path.exists(viirs_folder):
    os.makedirs(viirs_folder)

urbanD = gpd.read_file(urban_extents_file)
urbanHD = gpd.read_file(urban_extents_hd_file)

for ntl_file in ntl_files:
    name = ntl_file.split("/")[-1].split("_")[2][:8]
    inR = rasterio.open(ntl_file)
    tPrint("Processing %s" % name)
    urban_res_file = os.path.join(viirs_folder, f"URBAN_{name}.csv")
    urban_hd_res_file = os.path.join(viirs_folder, f"HD_URBAN_{name}.csv")
    admin_res_file = os.path.join(viirs_folder, f"ADMIN_{name}.csv")

    # Urban Summary
    if not os.path.exists(urban_res_file):
        urban_res = rMisc.zonalStats(urbanD, inR, minVal=0.1)
        col_names = [f"URBAN_{name}_{x}" for x in ["SUM", "MIN", "MAX", "MEAN"]]
        urban_df = pd.DataFrame(urban_res, columns=col_names)
        urban_df.to_csv(urban_res_file)
    # HD Urban Summary
    if not os.path.exists(urban_hd_res_file):
        hd_urban_res = rMisc.zonalStats(urbanHD, inR, minVal=0.1)
        col_names = [f"HD_URBAN_{name}_{x}" for x in ["SUM", "MIN", "MAX", "MEAN"]]
        hd_urban_df = pd.DataFrame(hd_urban_res, columns=col_names)
        hd_urban_df.to_csv(urban_hd_res_file)
    # admin Summary
    if not os.path.exists(admin_res_file):
        admin_res = rMisc.zonalStats(inAdmin, inR, minVal=0.1)
        col_names = [f"ADM_URBAN_{name}_{x}" for x in ["SUM", "MIN", "MAX", "MEAN"]]
        admin_df = pd.DataFrame(admin_res, columns=col_names)
        admin_df.to_csv(admin_res_file)

In [ ]:
# Compile VIIRS results
urb_files = [x for x in os.listdir(viirs_folder) if x.startswith("URBAN")]
for x in urb_files:
    tempD = pd.read_csv(os.path.join(viirs_folder, x), index_col=0)
    urbanD[x[:-4]] = tempD.iloc[:, 0]

hd_urb_files = [x for x in os.listdir(viirs_folder) if x.startswith("HD_URBAN")]
for x in hd_urb_files:
    tempD = pd.read_csv(os.path.join(viirs_folder, x), index_col=0)
    urbanHD[x[:-4]] = tempD.iloc[:, 0]

admin_urb_files = [x for x in os.listdir(viirs_folder) if x.startswith("ADMIN")]
for x in admin_urb_files:
    tempD = pd.read_csv(os.path.join(viirs_folder, x), index_col=0)
    inAdmin[x[:-4]] = tempD.iloc[:, 0]

urbanD.drop(["geometry"], axis=1).to_csv(urban_viirs_summary)
urbanHD.drop(["geometry"], axis=1).to_csv(urban_hd_viirs_summary)
inAdmin.drop(["geometry"], axis=1).to_csv(admin_viirs_summary)

# Summarize GHSL

In [ ]:
# List all files in ghsl folder
ghsl_files = [
    os.path.join(global_ghsl_folder, x)
    for x in os.listdir(global_ghsl_folder)
    if x.endswith(".tif")
]

for file_def in [
    # [admin_bounds, admin_ghsl_summary],
    [urban_extents_file, urban_ghsl_summary],
    [urban_extents_file, urban_ghsl_summary],
]:
    resG = gpd.read_file(file_def[0])

    for ghsl_file in ghsl_files:
        date = os.path.basename(ghsl_file).split("_")[3]
        inR = rasterio.open(ghsl_file)
        if resG.crs != inR.crs:
            resG = resG.to_crs(inR.crs)
        local_file = os.path.join(ghsl_folder, os.path.basename(ghsl_file))
        if not os.path.exists(local_file):
            rMisc.clipRaster(inR, resG, local_file)
        res = rMisc.zonalStats(resG, inR, minVal=0)
        res = pd.DataFrame(res, columns=["SUM", "MIN", "MAX", "MEAN"])
        resG[f"ghsl_{date}"] = res["SUM"]
        print(date)
    pd.DataFrame(resG.drop(["geometry"], axis=1)).to_csv(file_def[1])

# Join urban clusters and districts

In [ ]:
inUrban = gpd.read_file(urban_extents_file)
inHD = gpd.read_file(urban_extents_hd_file)
inAdmin = gpd.read_file(admin_bounds)
inAdmin = inAdmin.to_crs(inUrban.crs)

ntl_files = ntl.aws_search_ntl()

In [ ]:
def get_majority_polygon(
    shp, shp2, pop_layer, area_name="zz_area", zonal_sum_name="zz_sum"
):
    """Intersect shp(single polygon) with shp2(GeoDataFrame) to determine which row in shp2 has
        the highest zonal sum (ie - population)

    Args:
        shp: shapely polygon
        shp2: GeoDataFrame
        pop_layer: rasterio reader
    returns
        shp2 GeoDataFrame with two additional columns: area and zonal_sum
    """
    temp_shp = shp2.copy()
    for idx, row in temp_shp.iterrows():
        # Convert geometry in shp2 to the intersection with shp1
        xx = row["geometry"].intersection(shp.buffer(0)).buffer(0)
        temp_shp.loc[[idx], "geometry"] = gpd.GeoDataFrame(
            geometry=[xx]
        ).geometry.values

    # Run zonal analysis on pop_layer
    res = rMisc.zonalStats(temp_shp, pop_layer, reProj=True)
    res = pd.DataFrame(res, columns=["SUM", "MIN", "MAX", "MEAN"])

    shp2[zonal_sum_name] = res["SUM"].values
    shp2[area_name] = temp_shp["geometry"].apply(lambda x: x.area)

    return shp2.sort_values(zonal_sum_name, ascending=False)

In [ ]:
inUrban["HD_ID"] = ""
inUrban["Admin1_ID"] = ""
inUrban["Admin1_Pop"] = 0
inUrban["Admin2_ID"] = ""
inUrban["Admin2_Pop"] = 0


for idx, row in inUrban.iterrows():
    tPrint(idx)
    # Identify intersecting HD urban areas
    selHD = inHD.loc[inHD.intersects(row["geometry"])]
    if selHD.shape[0] == 1:
        inUrban.loc[idx, "HD_ID"] = selHD["ID"].iloc[0]
    elif selHD.shape[0] > 1:
        selHD = get_majority_polygon(row["geometry"], selHD, inP_ghs)
        inUrban.loc[idx, "HD_ID"] = selHD["ID"].iloc[0]

    # Identify intersecting admin areas
    selAdmin = inAdmin.loc[inAdmin.intersects(row["geometry"])]
    if selAdmin.shape[0] == 1:
        inUrban.loc[idx, "Admin1_ID"] = selAdmin["district_c"].iloc[0]
    elif selAdmin.shape[0] > 1:
        selAdmin = get_majority_polygon(row["geometry"], selAdmin, inP_ghs)
        inUrban.loc[idx, "Admin1_ID"] = selAdmin["district_c"].iloc[0]
        inUrban.loc[idx, "Admin1_Pop"] = selAdmin["zz_sum"].iloc[0]
        inUrban.loc[idx, "Admin2_ID"] = selAdmin["district_c"].iloc[1]
        inUrban.loc[idx, "Admin2_Pop"] = selAdmin["zz_sum"].iloc[1]

In [ ]:
inUrban.to_file()

In [ ]:
pd.DataFrame(inUrban.drop(["geometry"], axis=1)).to_csv(
    urban_extents_file.replace(".geojson", "_named.csv")
)

# Calculate admin centroids

In [ ]:
inAdmin_centroids = inAdmin.copy()
for idx, row in inAdmin.iterrows():
    # create numpy array of intersecting population layer
    curP, transform = mask(inR, [row.geometry], crop=True)
    curP[curP < 0] = 0
    centroid_coords = ndimage.center_of_mass(curP)

    b = row.geometry.bounds
    x_range = b[2] - b[0]
    y_range = b[3] - b[1]

    x_coord = b[0] + x_range * (centroid_coords[1] / curP.shape[1])
    y_coord = b[1] + y_range * (centroid_coords[2] / curP.shape[2])
    final_geom = Point(x_coord, y_coord)
    inAdmin_centroids.loc[idx, "geometry"] = final_geom
inAdmin_centroids.to_file(admin_final.replace(".shp", "_centroids.shp"))

# Calculate travel time

In [ ]:
if not os.path.exists(local_friction):
    globalP = rasterio.open(global_friction_surface)
    rMisc.clipRaster(globalP, inAdmin, local_friction)

dests = gpd.read_file(urban_extents_file)
dests["geometry"] = dests["geometry"].apply(lambda x: x.centroid)
inR = rasterio.open(local_friction)
frictionD = inR.read()[0, :, :]
frictionD = frictionD * 1000
mcp = graph.MCP_Geometric(frictionD)

In [ ]:
dests.head()

In [ ]:
importlib.reload(ma)
# Calculate travel time between all urban areas
all_rts = ma.get_linear_routes(inR, frictionD, dests, dests, "ID", "ID", verbose=True)
all_rts = all_rts.to_crs(3857)
all_rts["length_km"] = all_rts["geometry"].apply(lambda x: x.length / 1000)

In [ ]:
# all_rts.to_file(all_routes_file)
pd.pivot_table(all_rts, "cost", "origin", "destination").to_csv(time_matrix)
pd.pivot_table(all_rts, "length_km", "origin", "destination").to_csv(dist_matrix)

In [ ]:
inAdmin_centroids

In [ ]:
importlib.reload(ma)
# Calculate travel time between all urban areas and district centroids
inAdmin_centroids = gpd.read_file(admin_final.replace(".shp", "_centroids.shp"))
dist_all_rts = ma.get_linear_routes_mp(
    inR, frictionD, dests, inAdmin_centroids, "ID", "district_c", verbose=True
)
dist_all_rts = dist_all_rts.to_crs(3857)
dist_all_rts["length_km"] = dist_all_rts["geometry"].apply(lambda x: x.length / 1000)

In [ ]:
dist_all_rts.head()

In [ ]:
# dist_all_rts.to_file(dist_all_routes_file)
pd.pivot_table(dist_all_rts, "cost", "origin", "destination").to_csv(dist_time_matrix)
pd.pivot_table(dist_all_rts, "length_km", "origin", "destination").to_csv(
    dist_dist_matrix
)

In [ ]:
# For each urban area, generate a travel time to the centroid, and then sample for all the other areas
urban_output_matrix = np.zeros([dests.shape[0], dests.shape[0]])
for idx, row in dests.iterrows():
    costs, trace = ma.calculate_travel_time(inR, mcp, row.to_frame().transpose())
    cur_res = dests["geometry"].apply(lambda x: costs[inR.index(x.x, x.y)])
    output_matrix[idx,] = cur_res
    tPrint(f"{idx} of {dests.shape[0]} completed")

In [ ]:
tt_res = pd.DataFrame(output_matrix, columns=[f"urb_{x}" for x in dests["ID"]])
tt_res.index = dests["ID"]
tt_res.to_csv(urban_tt_result)

# attach IDs from ECA database to KAZ database

In [ ]:
urbanD = gpd.read_file(urban_extents_file)
eca_kaz = gpd.read_file(
    "/home/wb411133/projects/KAZ_SCADR_Urbanization/DATA/ECA_Extents/KAZ/KAZ_urban_extents.geojson"
)
urbanD["eca_id"] = 0
for idx, row in urbanD.iterrows():
    sel_eca = eca_kaz.loc[eca_kaz.intersects(row["geometry"].centroid)]
    if sel_eca.shape[0] > 0:
        urbanD.loc[idx, "eca_id"] = sel_eca["ID"].iloc[0]
urbanD.to_file(urban_extents_file, driver="GeoJSON")

In [ ]:
urbanHD = gpd.read_file(urban_extents_hd_file)
eca_kaz = gpd.read_file(
    "/home/wb411133/projects/KAZ_SCADR_Urbanization/DATA/ECA_Extents/KAZ/KAZ_urban_extents_hd.geojson"
)
urbanHD["eca_id"] = 0
for idx, row in urbanHD.iterrows():
    sel_eca = eca_kaz.loc[eca_kaz.intersects(row["geometry"].centroid)]
    if sel_eca.shape[0] > 0:
        urbanHD.loc[idx, "eca_id"] = sel_eca["ID"].iloc[0]
urbanHD.to_file(urban_extents_hd_file, driver="GeoJSON")

# DEBUGGING

In [ ]:
world_filepath = gpd.datasets.get_path("naturalearth_lowres")
world = gpd.read_file(world_filepath)
sel_country = world.loc[world["name"] == "Kenya"]

local_friction = "/home/wb411133/temp/KEN_friction.tif"
if not os.path.exists(local_friction):
    globalP = rasterio.open(global_friction_surface)
    rMisc.clipRaster(globalP, sel_country, local_friction)

inAdmin = gpd.read_file(
    "/home/public/Data/COUNTRY/KEN/ADMIN/geoBoundaries-KEN-ADM1.geojson"
)
inAdmin_centroids = inAdmin.copy()
inAdmin_centroids["geometry"] = inAdmin_centroids["geometry"].apply(
    lambda x: x.centroid
)

In [ ]:
inR = rasterio.open(local_friction)
frictionD = inR.read()[0, :, :]
frictionD = frictionD * 1000
mcp = graph.MCP_Geometric(frictionD)

In [ ]:
importlib.reload(ma)

all_rts = ma.get_linear_routes(
    inR,
    frictionD,
    inAdmin_centroids,
    inAdmin_centroids,
    "shapeName",
    "shapeName",
    verbose=True,
)
all_rts = all_rts.to_crs(3857)
all_rts["length_km"] = all_rts["geometry"].apply(lambda x: x.length / 1000)

In [ ]:
# Map resulting route
centre = sel_country.unary_union.centroid
m = folium.Map(location=[centre.y, centre.x], zoom_start=4)
orig_map = inAdmin_centroids.iloc[0]
rts = folium.GeoJson(
    mapping(all_rts.unary_union),
    style_function=lambda feature: {"color": "red", "weight": 1},
)

folium.CircleMarker(
    location=[orig_map.geometry.y, orig_map.geometry.x],
    radius=2,
    weight=4,
    color="blue",
).add_to(m)

rts.add_to(m)

m

In [ ]:
pd.pivot(all_rts, "origin", "destination", "cost")

In [ ]:
pd.pivot(all_rts, "origin", "destination", "length_km")

In [ ]:
# Generate h3 grid around dedicated city
m = folium.Map(location=[row.geometry.y, row.geometry.x], zoom_start=4)

folium.CircleMarker(
    location=[y_range[0], x_range[0]], radius=2, weight=4, color="red"
).add_to(m)

folium.CircleMarker(
    location=[y_range[0], x_range[-1]], radius=2, weight=4, color="blue"
).add_to(m)

folium.CircleMarker(
    location=[y_range[-1], x_range[-1]], radius=2, weight=4, color="orange"
).add_to(m)

folium.CircleMarker(
    location=[y_range[-1], x_range[0]], radius=2, weight=4, color="green"
).add_to(m)


m

In [ ]:
x_range[0]